In [1]:
import librosa
import soundfile as sf
import os, glob
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
def feature_extraction(audio_path, mfcc, chroma, mel):
    x , sr = librosa.load(audio_path)
    result=np.array([])
    
    if chroma:
        S=np.abs(librosa.stft(x)) 
        chroma = np.mean(librosa.feature.chroma_stft(S=S, sr=sr), axis=1)
        result=np.hstack((result, chroma))
        
    if mfcc:
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40), axis=1)
        result=np.hstack((result, mfcc))
        
    if mel:
        mfcc = np.mean(librosa.feature.melspectrogram(y=x, sr=sr), axis=1)
        result=np.hstack((result, mel))
          
    return result

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
def load_data(test_size):
    x, y =[], []
    
    for file in glob.glob("/Users/dhatrireddy/Desktop/data//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue 
        feature=feature_extraction(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
        

In [5]:
x_train,x_test,y_train,y_test = load_data(0.1)

In [6]:
print((x_train.shape[0], x_test.shape[0]))

(691, 77)


In [7]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 53


In [8]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [9]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [10]:
y_pred=model.predict(x_test)

In [11]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 80.52%
